# Converter for the Financial Filings at the Sec

In [1]:
from zipfile import ZipFile
import pandas as pd

dirname = "2024q2"
with ZipFile('importFiles/'+dirname+'.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

23899230
13918870
1058364
1157049
24168


In [2]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+-----------------------------------------+--------------+---------+----------+--------+-------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | adsh                 | tag                                     | version      |   coreg |    ddate |   qtrs | uom   |        value | footnote                                                                                                                                                                                                        

In [3]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+----------------------------------+-----------+-------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                              | version   | plabel                              |   negating |
|----+----------------------+----------+--------+--------+---------+---------+----------------------------------+-----------+-------------------------------------+------------|
|  0 | 0001654954-24-003968 |        1 |     37 | CP     |       0 | H       | AddressTypeDomain                | dei/2022  | AddressTypeDomain                   |          0 |
|  1 | 0001654954-24-004069 |        1 |     39 | CP     |       0 | H       | AddressTypeDomain                | dei/2022  | AddressTypeDomain                   |          0 |
|  2 | 0001654954-24-004067 |        1 |     45 | CP     |       0 | H       | AddressTypeDomain                | d

In [4]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------+-----------------------------------------+-------+-------------+----------+---------------------------+------------+------------------------------------------+-----------------------------------------+------------------+-------------+----------+---------------------------+------------+------------------------------------------+--------------------------------+--------------+-----------+-----------+----------------------------------------------------+---------------+-------+--------+-------+---------+-------------+------+------+----------+-----------------------+-----------+----------+-------------------------------+---------+---------------------------------+
|    | adsh                 |     cik | name                                    |   sic | countryba   | stprba   | cityba                    | zipba      | bas1                                     | bas2                                    | baph             | countryma   | stprma   | cityma  

In [5]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------+----------+------------+----------------------+--------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                                                                                                        

In [6]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
coreg        object
ddate         int64
qtrs          int64
uom          object
value       float64
footnote     object
dtype: object


In [7]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh        0001193125-24-163247
tag                   SharePrice
version             us-gaap/2023
coreg                        NaN
ddate                   20210630
qtrs                           0
uom                          USD
value                       9.71
footnote                     NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [8]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [9]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [10]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 425 Dtos size: 164 Bs: 74 Cf: 90 Ic: 0 Json size: 46kb.
file 10 of 29400 exportFiles/2024q2/0000726958-24-000046.json stored in 12152.877ms.
FilterDfNum size: 864 Dtos size: 236 Bs: 101 Cf: 135 Ic: 0 Json size: 60kb.
file 30 of 29400 exportFiles/2024q2/0000950170-24-076649.json stored in 26029.346ms.
FilterDfNum size: 474 Dtos size: 160 Bs: 83 Cf: 77 Ic: 0 Json size: 54kb.
file 44 of 29400 exportFiles/2024q2/0000950170-24-076842.json stored in 14086.481ms.
FilterDfNum size: 343 Dtos size: 127 Bs: 59 Cf: 68 Ic: 0 Json size: 31kb.
file 55 of 29400 exportFiles/2024q2/0001062993-24-012996.json stored in 9903.771ms.
FilterDfNum size: 2 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 90 of 29400 exportFiles/2024q2/0001130713-24-000047.json stored in 301.743ms.
FilterDfNum size: 338 Dtos size: 120 Bs: 68 Cf: 52 Ic: 0 Json size: 35kb.
file 99 of 29400 exportFiles/2024q2/0001185185-24-000650.json stored in 9633.128ms.
File exportFiles/2024q2/0001193125-24-166395.json has P

FilterDfNum size: 434 Dtos size: 187 Bs: 68 Cf: 119 Ic: 0 Json size: 55kb.
file 697 of 29400 exportFiles/2024q2/0001437749-24-021311.json stored in 14605.882ms.
FilterDfNum size: 325 Dtos size: 143 Bs: 77 Cf: 66 Ic: 0 Json size: 38kb.
file 700 of 29400 exportFiles/2024q2/0001477932-24-003883.json stored in 10828.946ms.
FilterDfNum size: 85 Dtos size: 49 Bs: 36 Cf: 13 Ic: 0 Json size: 13kb.
file 710 of 29400 exportFiles/2024q2/0001493152-24-025203.json stored in 3035.036ms.
FilterDfNum size: 317 Dtos size: 124 Bs: 65 Cf: 59 Ic: 0 Json size: 37kb.
file 716 of 29400 exportFiles/2024q2/0001493152-24-025217.json stored in 10438.635ms.
FilterDfNum size: 209 Dtos size: 93 Bs: 58 Cf: 35 Ic: 0 Json size: 26kb.
file 718 of 29400 exportFiles/2024q2/0001493152-24-025250.json stored in 6827.105ms.
File exportFiles/2024q2/0001628280-24-030179.json has Period: nan
File exportFiles/2024q2/0001628280-24-030200.json has Period: nan
FilterDfNum size: 114 Dtos size: 62 Bs: 42 Cf: 20 Ic: 0 Json size: 17kb.

FilterDfNum size: 407 Dtos size: 128 Bs: 56 Cf: 72 Ic: 0 Json size: 35kb.
file 1099 of 29400 exportFiles/2024q2/0001702744-24-000045.json stored in 13417.373ms.
FilterDfNum size: 124 Dtos size: 64 Bs: 48 Cf: 16 Ic: 0 Json size: 18kb.
file 1106 of 29400 exportFiles/2024q2/0001829126-24-004446.json stored in 4141.666ms.
FilterDfNum size: 456 Dtos size: 134 Bs: 82 Cf: 52 Ic: 0 Json size: 37kb.
file 1125 of 29400 exportFiles/2024q2/0000055529-24-000021.json stored in 14976.884ms.
FilterDfNum size: 1359 Dtos size: 620 Bs: 325 Cf: 295 Ic: 0 Json size: 169kb.
file 1131 of 29400 exportFiles/2024q2/0000709283-24-000014.json stored in 44419.197ms.
FilterDfNum size: 2 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 1178 of 29400 exportFiles/2024q2/0001062993-24-013109.json stored in 320.25ms.
FilterDfNum size: 353 Dtos size: 103 Bs: 54 Cf: 49 Ic: 0 Json size: 27kb.
file 1215 of 29400 exportFiles/2024q2/0001104659-24-076291.json stored in 11495.896ms.
FilterDfNum size: 2 Dtos size: 0 Bs: 0 Cf:

FilterDfNum size: 424 Dtos size: 178 Bs: 94 Cf: 84 Ic: 0 Json size: 49kb.
file 1791 of 29400 exportFiles/2024q2/0000107687-24-000017.json stored in 14722.389ms.
File exportFiles/2024q2/0000864270-24-000003.json has Symbol Series([], )
FilterDfNum size: 546 Dtos size: 188 Bs: 77 Cf: 111 Ic: 0 Json size: 52kb.
file 1826 of 29400 exportFiles/2024q2/0000950170-24-075584.json stored in 19083.643ms.
FilterDfNum size: 704 Dtos size: 160 Bs: 73 Cf: 87 Ic: 0 Json size: 46kb.
file 1830 of 29400 exportFiles/2024q2/0000950170-24-075605.json stored in 24881.684ms.
FilterDfNum size: 229 Dtos size: 72 Bs: 40 Cf: 32 Ic: 0 Json size: 19kb.
file 1838 of 29400 exportFiles/2024q2/0001017386-24-000190.json stored in 8228.909ms.
FilterDfNum size: 219 Dtos size: 86 Bs: 48 Cf: 38 Ic: 0 Json size: 23kb.
file 1848 of 29400 exportFiles/2024q2/0001079973-24-000912.json stored in 7714.731ms.
FilterDfNum size: 198 Dtos size: 92 Bs: 58 Cf: 34 Ic: 0 Json size: 27kb.
file 1850 of 29400 exportFiles/2024q2/0001096906-24

FilterDfNum size: 441 Dtos size: 139 Bs: 74 Cf: 65 Ic: 0 Json size: 37kb.
file 2503 of 29400 exportFiles/2024q2/0000950170-24-074297.json stored in 14111.21ms.
FilterDfNum size: 126 Dtos size: 52 Bs: 34 Cf: 18 Ic: 0 Json size: 15kb.
file 2577 of 29400 exportFiles/2024q2/0001213900-24-053108.json stored in 4169.153ms.
File exportFiles/2024q2/0001213900-24-053125.json has Period: nan
FilterDfNum size: 352 Dtos size: 106 Bs: 62 Cf: 44 Ic: 0 Json size: 29kb.
file 2583 of 29400 exportFiles/2024q2/0001213900-24-053154.json stored in 11068.975ms.
FilterDfNum size: 227 Dtos size: 67 Bs: 35 Cf: 32 Ic: 0 Json size: 18kb.
file 2585 of 29400 exportFiles/2024q2/0001213900-24-053321.json stored in 7049.213ms.
FilterDfNum size: 362 Dtos size: 92 Bs: 35 Cf: 57 Ic: 0 Json size: 24kb.
file 2588 of 29400 exportFiles/2024q2/0001213900-24-053334.json stored in 10995.267ms.
FilterDfNum size: 312 Dtos size: 105 Bs: 42 Cf: 63 Ic: 0 Json size: 29kb.
file 2598 of 29400 exportFiles/2024q2/0001213900-24-053407.js

...

FilterDfNum size: 348 Dtos size: 130 Bs: 62 Cf: 68 Ic: 0 Json size: 34kb.
file 28968 of 29400 exportFiles/2024q2/0001213900-24-028218.json stored in 11360.478ms.
FilterDfNum size: 366 Dtos size: 130 Bs: 76 Cf: 54 Ic: 0 Json size: 36kb.
file 28969 of 29400 exportFiles/2024q2/0001213900-24-028228.json stored in 11912.754ms.
FilterDfNum size: 547 Dtos size: 154 Bs: 80 Cf: 74 Ic: 0 Json size: 42kb.
file 28973 of 29400 exportFiles/2024q2/0001213900-24-028244.json stored in 17977.572ms.
FilterDfNum size: 405 Dtos size: 138 Bs: 52 Cf: 86 Ic: 0 Json size: 38kb.
file 28974 of 29400 exportFiles/2024q2/0001213900-24-028253.json stored in 13129.3ms.
FilterDfNum size: 2 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 28975 of 29400 exportFiles/2024q2/0001213900-24-028258.json stored in 291.12ms.
File exportFiles/2024q2/0001213900-24-028261.json has Period: nan
File exportFiles/2024q2/0001213900-24-028274.json has Symbol Series([], )
FilterDfNum size: 250 Dtos size: 83 Bs: 54 Cf: 29 Ic: 0 Json s

FilterDfNum size: 165 Dtos size: 89 Bs: 46 Cf: 43 Ic: 0 Json size: 38kb.
file 29063 of 29400 exportFiles/2024q2/0001410578-24-000372.json stored in 5491.573ms.
FilterDfNum size: 121 Dtos size: 63 Bs: 37 Cf: 26 Ic: 0 Json size: 30kb.
file 29064 of 29400 exportFiles/2024q2/0001410578-24-000374.json stored in 4102.612ms.
FilterDfNum size: 405 Dtos size: 154 Bs: 73 Cf: 81 Ic: 0 Json size: 41kb.
file 29065 of 29400 exportFiles/2024q2/0001410578-24-000381.json stored in 12917.934ms.
FilterDfNum size: 183 Dtos size: 91 Bs: 57 Cf: 34 Ic: 0 Json size: 25kb.
file 29066 of 29400 exportFiles/2024q2/0001410578-24-000385.json stored in 5852.549ms.
FilterDfNum size: 375 Dtos size: 124 Bs: 67 Cf: 57 Ic: 0 Json size: 33kb.
file 29067 of 29400 exportFiles/2024q2/0001410578-24-000401.json stored in 12244.188ms.
FilterDfNum size: 189 Dtos size: 78 Bs: 42 Cf: 36 Ic: 0 Json size: 28kb.
file 29068 of 29400 exportFiles/2024q2/0001410578-24-000408.json stored in 6596.138ms.
FilterDfNum size: 234 Dtos size: 96 

FilterDfNum size: 350 Dtos size: 125 Bs: 70 Cf: 55 Ic: 0 Json size: 36kb.
file 29148 of 29400 exportFiles/2024q2/0001493152-24-012145.json stored in 11476.816ms.
FilterDfNum size: 263 Dtos size: 97 Bs: 59 Cf: 38 Ic: 0 Json size: 35kb.
file 29150 of 29400 exportFiles/2024q2/0001493152-24-012159.json stored in 8386.966ms.
FilterDfNum size: 320 Dtos size: 123 Bs: 64 Cf: 59 Ic: 0 Json size: 35kb.
file 29154 of 29400 exportFiles/2024q2/0001493152-24-012177.json stored in 10555.529ms.
FilterDfNum size: 162 Dtos size: 65 Bs: 40 Cf: 25 Ic: 0 Json size: 18kb.
file 29156 of 29400 exportFiles/2024q2/0001493152-24-012185.json stored in 5463.902ms.
FilterDfNum size: 308 Dtos size: 144 Bs: 68 Cf: 76 Ic: 0 Json size: 40kb.
file 29159 of 29400 exportFiles/2024q2/0001493152-24-012225.json stored in 10306.872ms.
FilterDfNum size: 342 Dtos size: 112 Bs: 55 Cf: 57 Ic: 0 Json size: 36kb.
file 29161 of 29400 exportFiles/2024q2/0001493152-24-012233.json stored in 11201.366ms.
FilterDfNum size: 201 Dtos size:

FilterDfNum size: 114 Dtos size: 56 Bs: 35 Cf: 21 Ic: 0 Json size: 16kb.
file 29230 of 29400 exportFiles/2024q2/0001493152-24-012498.json stored in 3941.399ms.
FilterDfNum size: 426 Dtos size: 151 Bs: 71 Cf: 80 Ic: 0 Json size: 43kb.
file 29232 of 29400 exportFiles/2024q2/0001493152-24-012503.json stored in 13905.408ms.
FilterDfNum size: 306 Dtos size: 136 Bs: 68 Cf: 68 Ic: 0 Json size: 37kb.
file 29233 of 29400 exportFiles/2024q2/0001493152-24-012505.json stored in 10275.292ms.
File exportFiles/2024q2/0001493152-24-012508.json has Symbol Series([], )
FilterDfNum size: 503 Dtos size: 172 Bs: 91 Cf: 81 Ic: 0 Json size: 47kb.
file 29235 of 29400 exportFiles/2024q2/0001493152-24-012509.json stored in 16421.912ms.
FilterDfNum size: 239 Dtos size: 90 Bs: 54 Cf: 36 Ic: 0 Json size: 25kb.
file 29237 of 29400 exportFiles/2024q2/0001493152-24-012533.json stored in 7931.757ms.
FilterDfNum size: 1 Dtos size: 0 Bs: 0 Cf: 0 Ic: 0 Json size: 0kb.
file 29238 of 29400 exportFiles/2024q2/0001493152-24-

FilterDfNum size: 628 Dtos size: 198 Bs: 54 Cf: 144 Ic: 0 Json size: 63kb.
file 29337 of 29400 exportFiles/2024q2/0001676238-24-000049.json stored in 20328.626ms.
FilterDfNum size: 167 Dtos size: 85 Bs: 48 Cf: 37 Ic: 0 Json size: 23kb.
file 29339 of 29400 exportFiles/2024q2/0001683168-24-001970.json stored in 5578.828ms.
File exportFiles/2024q2/0001683168-24-001989.json has Symbol Series([], )
FilterDfNum size: 274 Dtos size: 132 Bs: 70 Cf: 62 Ic: 0 Json size: 35kb.
file 29342 of 29400 exportFiles/2024q2/0001683168-24-001995.json stored in 8792.142ms.
FilterDfNum size: 365 Dtos size: 150 Bs: 82 Cf: 68 Ic: 0 Json size: 38kb.
file 29343 of 29400 exportFiles/2024q2/0001683168-24-002011.json stored in 11825.445ms.
FilterDfNum size: 178 Dtos size: 93 Bs: 53 Cf: 40 Ic: 0 Json size: 42kb.
file 29344 of 29400 exportFiles/2024q2/0001683168-24-002016.json stored in 6073.003ms.
FilterDfNum size: 255 Dtos size: 158 Bs: 78 Cf: 80 Ic: 0 Json size: 39kb.
file 29345 of 29400 exportFiles/2024q2/0001683